In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Libraries for processing sounds
import librosa
from IPython.display import Audio
import random

def extract_mel_spectrogram(file_path, sr=22050, n_mels=128):
    y, sr = librosa.load(file_path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db

def pad_or_trim(mel, target_width=400):
    if mel.shape[1] > target_width:
        mel = mel[:, :target_width]
    elif mel.shape[1] < target_width:
        pad_width = target_width - mel.shape[1]
        mel = np.pad(mel, ((0, 0), (0, pad_width)), mode='constant')
    return mel

class AnimalSoundDataset(Dataset):
    def __init__(self, root_dir, transform=None, split='train', split_ratio=0.8, seed=42):
        self.root_dir = root_dir
        self.classes = sorted(os.listdir(root_dir))
        self.transform = transform

        all_paths = []
        all_labels = []

        for idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_path):
                if file_name.endswith(".wav"):
                    all_paths.append(os.path.join(class_path, file_name))
                    all_labels.append(idx)

        # Shuffle and split
        combined = list(zip(all_paths, all_labels))
        random.seed(seed)
        random.shuffle(combined)
        split_point = int(len(combined) * split_ratio)

        if split == 'train':
            selected = combined[:split_point]
        elif split == 'val':
            selected = combined[split_point:]
        else:
            raise ValueError("split must be 'train' or 'val'")

        self.file_paths, self.labels = zip(*selected) if selected else ([], [])
    
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        mel = extract_mel_spectrogram(self.file_paths[idx])  # [n_mels, time]
        mel = pad_or_trim(mel, target_width=400)
        mel = torch.tensor(mel, dtype=torch.float32).unsqueeze(0)  # [1, H, W]
        label = self.labels[idx]
        return mel, label
    
        
    def get_class(self,idx):
        label = self.labels[idx]
        return self.classes[label]
    
    def visualize(self,n):
    #print(dataset[n][0].squeeze(0))
        plt.figure(figsize=(16,6))
        librosa.display.specshow(
                            self[n][0].squeeze(0).numpy(),
                            x_axis="time",
                            y_axis="mel")
        plt.colorbar()

    def play(self,n):
        path = self.file_paths[n]
        #print(path)
        x, Fs = librosa.load(path, sr=None)
        label = self.get_class(n)
        print('Class: {}'.format(label))
        return Audio(x, rate=Fs)

In [2]:
def change_path(path, new_path):
    train_csv = pd.read_csv(path)
    train_csv['path']=new_path
    train_csv['class'] = train_csv['name'].apply(lambda x: x.split('_')[0])
    train_csv.to_csv('data.csv')
    
    
    
path = 'Animal_Sound.csv'
new_path = 'Animal-Soundprepros/'
change_path(path, new_path=new_path)

train_csv = pd.read_csv(path)

dataset_train = AnimalSoundDataset(new_path, split='train', split_ratio=0.8, seed=42)
dataset_val = AnimalSoundDataset(new_path, split='val', split_ratio=0.8, seed=42)

x_train_list = []
y_train_list = []

loader = DataLoader(dataset_train, batch_size=len(dataset_train))
x_train, y_train = next(iter(loader))

loader = DataLoader(dataset_val, batch_size=len(dataset_val))
x_val, y_val = next(iter(loader))

In [3]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)


torch.Size([520, 1, 128, 400])
torch.Size([520])
torch.Size([130, 1, 128, 400])
torch.Size([130])


In [4]:
def extract_log_mel_spectrogram(
    filepath,
    sr=22050,
    n_fft=1024,
    hop_length=512,
    n_mels=128
):
    try:
        y, _ = librosa.load(filepath, sr=sr)

        # Check if audio is silent or invalid
        if not np.isfinite(y).all() or np.max(np.abs(y)) == 0:
            raise ValueError("Invalid or silent audio")

        # Normalize safely
        y = y / np.max(np.abs(y))

        # Compute mel spectrogram
        mel = librosa.feature.melspectrogram(
            y=y,
            sr=sr,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
        )

        # Convert to log scale
        log_mel = librosa.power_to_db(mel, ref=np.max)

        # Final safety check
        if not np.isfinite(log_mel).all():
            raise ValueError("NaN or inf in spectrogram")

        return log_mel

    except Exception as e:
        print(f"[WARNING] Failed to extract mel from {filepath}: {e}")
        # Return a zero tensor of expected shape
        return np.zeros((n_mels, 400), dtype=np.float32)


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Libraries for processing sounds
import librosa
from IPython.display import Audio
import random


def extract_mel_spectrogram(file_path, sr=22050, n_mels=128):
    y, sr = librosa.load(file_path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db

def pad_or_trim(mel, target_width=400):
    if mel.shape[1] > target_width:
        mel = mel[:, :target_width]
    elif mel.shape[1] < target_width:
        pad_width = target_width - mel.shape[1]
        mel = np.pad(mel, ((0, 0), (0, pad_width)), mode='constant')
    return mel

class AnimalSoundDataset_new(Dataset):
    def __init__(self, data_path, transform=None, split='train', split_ratio=0.8, seed=42):
        self.transform = transform
        self.data_path = data_path

        df = pd.read_csv(data_path)

        all_paths = df['path']
        all_labels = df['name']
        self.classes = df['name'].unique().tolist()

        # Build label-to-index map
        self.class_to_idx = {label: idx for idx, label in enumerate(sorted(set(all_labels)))}
        self.classes = list(self.class_to_idx.keys())  # e.g., ['cat', 'cow', 'dog']

        # Encode labels
        encoded_labels = [self.class_to_idx[label] for label in all_labels]

        # Shuffle and split
        combined = list(zip(all_paths, encoded_labels))
        random.seed(seed)
        random.shuffle(combined)
        split_point = int(len(combined) * split_ratio)

        if split == 'train':
            selected = combined[:split_point]
        elif split == 'val':
            selected = combined[split_point:]
        else:
            raise ValueError("split must be 'train' or 'val'")

        self.file_paths, self.labels = zip(*selected) if selected else ([], [])

    
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # mel = extract_mel_spectrogram(self.file_paths[idx])  # [n_mels, time]
        # mel = pad_or_trim(mel, target_width=400)
        # mel = torch.tensor(mel, dtype=torch.float32).unsqueeze(0)  # [1, H, W]
        # label = self.labels[idx]
        # return mel, label

        mel_log = extract_log_mel_spectrogram(self.file_paths[idx])  # [n_mels, time]
        mel_log = pad_or_trim(mel_log, target_width=400)
        mel_log = torch.tensor(mel_log, dtype=torch.float32).unsqueeze(0)
        label = self.labels[idx]
        return mel_log, label
  
    def get_class(self,idx):
        label = self.labels[idx]
        return self.classes[label]
    
    def visualize(self,n):
    #print(dataset[n][0].squeeze(0))
        plt.figure(figsize=(16,6))
        librosa.display.specshow(
                            self[n][0].squeeze(0).numpy(),
                            x_axis="time",
                            y_axis="mel")
        plt.colorbar()

    def play(self,n):
        path = self.file_paths[n]
        #print(path)
        x, Fs = librosa.load(path, sr=None)
        label = self.get_class(n)
        print('Class: {}'.format(label))
        return Audio(x, rate=Fs)

In [6]:
data_path = 'data/Animal_Sound_modified.csv'
dataset_train = AnimalSoundDataset_new(data_path, split='train', split_ratio=0.8, seed=42)
dataset_val = AnimalSoundDataset_new(data_path, split='val', split_ratio=0.8, seed=42)

x_train_list = []
y_train_list = []

loader = DataLoader(dataset_train, batch_size=len(dataset_train))
x_train, y_train = next(iter(loader))

loader = DataLoader(dataset_val, batch_size=len(dataset_val))
x_val, y_val = next(iter(loader))

[WARNING] Failed to extract mel from data/augmented/Lion_1_modified.wav: Invalid or silent audio
[WARNING] Failed to extract mel from data/sounds/Lion_1.wav: zero-size array to reduction operation maximum which has no identity


In [7]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)


torch.Size([1040, 1, 128, 400])
torch.Size([1040])
torch.Size([260, 1, 128, 400])
torch.Size([260])
